In [10]:
import argparse
import logging
import numpy as np
from time import time
import utils as U
import codecs

In [11]:
logging.basicConfig(
    # filename='out.log',
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(message)s')
logger = logging.getLogger(__name__)

In [12]:
# Parse arguments

parser = argparse.ArgumentParser()
parser.add_argument("-o", "--out-dir", dest="out_dir_path", type=str, metavar='<str>', default='output',
                    help="The path to the output directory")
parser.add_argument("-e", "--embdim", dest="emb_dim", type=int, metavar='<int>', default=200,
                    help="Embeddings dimension (default=200)")
parser.add_argument("-b", "--batch-size", dest="batch_size", type=int, metavar='<int>', default=50,
                    help="Batch size (default=50)")
parser.add_argument("-v", "--vocab-size", dest="vocab_size", type=int, metavar='<int>', default=2000,
                    help="Vocab size. '0' means no limit (default=1000)")
parser.add_argument("-as", "--aspect-size", dest="aspect_size", type=int, metavar='<int>', default=1,
                    help="The number of aspects specified by users (default=14)")
parser.add_argument("--emb", dest="emb_path", type=str, metavar='<str>', help="The path to the word embeddings file",default='preprocessed_data/w2v_embedding')
parser.add_argument("--epochs", dest="epochs", type=int, metavar='<int>', default=15,
                    help="Number of epochs (default=15)")
parser.add_argument("-n", "--neg-size", dest="neg_size", type=int, metavar='<int>', default=20,
                    help="Number of negative instances (default=20)")
parser.add_argument("--maxlen", dest="maxlen", type=int, metavar='<int>', default=0,
                    help="Maximum allowed number of words during training. '0' means no limit (default=0)")
parser.add_argument("--seed", dest="seed", type=int, metavar='<int>', default=1234, help="Random seed (default=1234)")
parser.add_argument("-a", "--algorithm", dest="algorithm", type=str, metavar='<str>', default='adam',
                    help="Optimization algorithm (rmsprop|sgd|adagrad|adadelta|adam|adamax) (default=adam)")
parser.add_argument("--domain", dest="domain", type=str, metavar='<str>', default='restaurant',
                    help="domain of the corpus {restaurant, beer}")
parser.add_argument("--ortho-reg", dest="ortho_reg", type=float, metavar='<float>', default=0.1,
                    help="The weight of orthogonol regularizaiton (default=0.1)")

_StoreAction(option_strings=['--ortho-reg'], dest='ortho_reg', nargs=None, const=None, default=0.1, type=<class 'float'>, choices=None, help='The weight of orthogonol regularizaiton (default=0.1)', metavar='<float>')

In [13]:
args = parser.parse_args(args=[])

In [14]:
out_dir = args.out_dir_path 
U.mkdir_p(out_dir)
U.print_args(args)

I0912 08:36:03.490820  8168 utils.py:142] Arguments:
I0912 08:36:03.491819  8168 utils.py:149]   algorithm: adam
I0912 08:36:03.492819  8168 utils.py:149]   aspect_size: 1
I0912 08:36:03.492819  8168 utils.py:149]   batch_size: 50
I0912 08:36:03.493817  8168 utils.py:149]   command: C:\Users\t15k\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py -f C:\Users\t15k\AppData\Roaming\jupyter\runtime\kernel-80391531-7ea8-4cad-bded-8f3f1e0c1747.json
I0912 08:36:03.494818  8168 utils.py:149]   domain: restaurant
I0912 08:36:03.495819  8168 utils.py:149]   emb_dim: 200
I0912 08:36:03.495819  8168 utils.py:149]   emb_path: preprocessed_data/w2v_embedding
I0912 08:36:03.496819  8168 utils.py:149]   epochs: 15
I0912 08:36:03.497818  8168 utils.py:149]   maxlen: 0
I0912 08:36:03.498819  8168 utils.py:149]   neg_size: 20
I0912 08:36:03.498819  8168 utils.py:149]   ortho_reg: 0.1
I0912 08:36:03.499819  8168 utils.py:149]   out_dir_path: output
I0912 08:36:03.500818  8168 utils.

In [15]:
if args.seed > 0:
    np.random.seed(args.seed)

In [16]:
# ###############################################################################################################################
# ## Prepare data
# #

from keras.preprocessing import sequence
import reader as dataset

vocab, train_x, test_x, overall_maxlen = dataset.get_data(vocab_size=args.vocab_size, maxlen=args.maxlen)

 Creating vocab ...
   7710 total words, 1658 unique words
  keep the top 2000 words
 Reading dataset ...
  train set
   <num> hit rate: 0.00%, <unk> hit rate: 0.00%
  test set
   <num> hit rate: 0.00%, <unk> hit rate: 11.69%


In [17]:
train_x = sequence.pad_sequences(train_x, maxlen=overall_maxlen)
test_x = sequence.pad_sequences(test_x, maxlen=overall_maxlen)

In [18]:
print('Number of training examples: ', len(train_x))
print('Length of vocab: ', len(vocab))

Number of training examples:  580
Length of vocab:  1661


In [19]:
def sentence_batch_generator(data, batch_size):
    n_batch = len(data) // batch_size
    batch_count = 0
    np.random.shuffle(data)

    while True:
        if batch_count >= n_batch:
            np.random.shuffle(data)
            batch_count = 0

        batch = data[batch_count * batch_size: (batch_count + 1) * batch_size]
        batch_count += 1
        yield batch

In [20]:
def negative_batch_generator(data, batch_size, neg_size):
    data_len = data.shape[0]
    dim = data.shape[1]

    while True:
        indices = np.random.choice(data_len, batch_size * neg_size)
        samples = data[indices].reshape(batch_size, neg_size, dim)
        yield samples

In [21]:
# Optimizer algorithm --------------------------------------------------------------------------------------------------

from optimizers import get_optimizer

optimizer = get_optimizer(args)

In [22]:
# Building model -------------------------------------------------------------------------------------------------------

from model import create_model
import keras.backend as K

logger.info('  Building model')

I0912 08:36:17.802223  8168 textcleaner.py:37] 'pattern' package not found; tag filters are not available for English
I0912 08:36:20.436011  8168 <ipython-input-22-a519dc19c43c>:6]   Building model


In [23]:
def max_margin_loss(y_true, y_pred):
    return K.mean(y_pred)

In [24]:
model = create_model(args, overall_maxlen, vocab)

W0912 08:36:34.536604  8168 deprecation_wrapper.py:119] From C:\Users\t15k\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0912 08:36:34.543597  8168 deprecation_wrapper.py:119] From C:\Users\t15k\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4420: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0912 08:36:34.583597  8168 deprecation_wrapper.py:119] From C:\Users\t15k\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

I0912 08:36:34.768606  8168 w2v_emb_reader.py:22] Loading embeddings from: preprocessed_data/w2v_embedding
I0912 08:36:34.769620  8168 utils.py:422] loading Word2Vec object from preprocessed_data/w2v_embedding
I0912 08:36:34.774598  

In [25]:
# freeze the word embedding layer
model.get_layer('word_emb').trainable = False
model.compile(optimizer=optimizer, loss=max_margin_loss, metrics=[max_margin_loss])


W0912 08:36:35.543277  8168 deprecation_wrapper.py:119] From C:\Users\t15k\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [26]:
## Training
#
from tqdm import tqdm

logger.info(
    '--------------------------------------------------------------------------------------------------------------------------')

vocab_inv = {}

for w, ind in vocab.items():
    vocab_inv[ind] = w

I0912 08:36:36.538443  8168 <ipython-input-26-d7122f67df2c>:6] --------------------------------------------------------------------------------------------------------------------------


In [27]:
sen_gen = sentence_batch_generator(train_x, args.batch_size)
neg_gen = negative_batch_generator(train_x, args.batch_size, args.neg_size)
batches_per_epoch = len(train_x) // args.batch_size

In [28]:
print("Batches per epoch", batches_per_epoch)

min_loss = float('inf')

Batches per epoch 11


In [29]:
#for ii in range(args.epochs):
for ii in range(20):
    t0 = time()
    loss, max_margin_loss = 0., 0.

    for b in tqdm(range(batches_per_epoch)):
        sen_input = next(sen_gen)
        neg_input = next(neg_gen)

        try:
            batch_loss, batch_max_margin_loss = model.train_on_batch([sen_input, neg_input], np.ones((args.batch_size, 1)))
        except Exception as e:
            print(e)
            print(sen_input.shape, sen_input)
            print(neg_input.shape, neg_input)

            print()
            quit()

        loss += batch_loss / batches_per_epoch
        max_margin_loss += batch_max_margin_loss / batches_per_epoch

    tr_time = time() - t0

    if loss < min_loss:

        min_loss = loss
        word_emb = K.get_value(model.get_layer('word_emb').embeddings)
        aspect_emb = K.get_value(model.get_layer('aspect_emb').W)
        word_emb = word_emb / np.linalg.norm(word_emb, axis=-1, keepdims=True)
        aspect_emb = aspect_emb / np.linalg.norm(aspect_emb, axis=-1, keepdims=True)
        aspect_file = codecs.open(out_dir + '/aspect.log', 'w', 'utf-8')
        model.save_weights(out_dir + '/model_param')

        for ind in range(len(aspect_emb)):
            desc = aspect_emb[ind]
            sims = word_emb.dot(desc.T)
            ordered_words = np.argsort(sims)[::-1]
            desc_list = [vocab_inv[w] + " : " + str(sims[w]) for w in ordered_words[:20]]
            desc_list_words = [vocab_inv[w] for w in ordered_words[:20]]
            print('Aspect %d:' % ind)
            print(desc_list)
            aspect_file.write('Aspect %d:\n' % ind)
            aspect_file.write(' '.join(desc_list) + '\n\n')

    logger.info('Epoch %d, train: %is' % (ii, tr_time))
    logger.info(
        'Total loss: %.4f, max_margin_loss: %.4f, ortho_reg: %.4f' % (loss, max_margin_loss, loss - max_margin_loss))

  0%|                                                                                           | 0/11 [00:00<?, ?it/s]W0912 08:36:40.611649  8168 deprecation.py:323] From C:\Users\t15k\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:02<00:00,  1.08s/it]


Aspect 0:
['בזק : 0.9988861', 'ממשלה : 0.9988583', 'שאול : 0.9988226', "אלוביץ' : 0.99881405", 'שוחד : 0.9986738', 'ראש : 0.9986363', 'תיק : 0.9986132', 'נתניהו : 0.99860847', 'החלטה : 0.9984477', 'שליטה : 0.9984382', 'דין : 0.99842376', 'אביחי : 0.998381', 'בעל : 0.99837226', 'אישום : 0.99833715', 'יועץ : 0.99833244', 'הפרה : 0.99833226', 'מנדלבליט : 0.99830604', 'רשות : 0.9982872', 'משפטי : 0.99827975', 'כתב : 0.9982499']


I0912 08:36:42.921411  8168 <ipython-input-29-f8eef542f0f5>:46] Epoch 0, train: 2s
I0912 08:36:42.923413  8168 <ipython-input-29-f8eef542f0f5>:48] Total loss: 19.5420, max_margin_loss: 19.5420, ortho_reg: 0.0000
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 44.00it/s]


Aspect 0:
['בזק : 0.998527', 'שאול : 0.9985069', "אלוביץ' : 0.9984623", 'ממשלה : 0.99837315', 'תיק : 0.9982135', 'נתניהו : 0.99819726', 'ראש : 0.9981855', 'שוחד : 0.9981682', 'הפרה : 0.99813384', 'אביחי : 0.99810433', 'דין : 0.99805117', 'משפטי : 0.99802876', 'שליטה : 0.99799806', 'החלטה : 0.99795294', 'בעל : 0.9979458', 'מנדלבליט : 0.9979377', 'יועץ : 0.99793017', 'אמון : 0.99792576', 'כתב : 0.99792075', 'רשות : 0.99782014']


I0912 08:36:43.226412  8168 <ipython-input-29-f8eef542f0f5>:46] Epoch 1, train: 0s
I0912 08:36:43.227415  8168 <ipython-input-29-f8eef542f0f5>:48] Total loss: 19.0219, max_margin_loss: 19.0219, ortho_reg: 0.0000
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 45.64it/s]


...

I0912 08:36:45.346423  8168 <ipython-input-29-f8eef542f0f5>:46] Epoch 8, train: 0s
I0912 08:36:45.347424  8168 <ipython-input-29-f8eef542f0f5>:48] Total loss: 18.5309, max_margin_loss: 18.5309, ortho_reg: -0.0000
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 42.31it/s]
I0912 08:36:45.612425  8168 <ipython-input-29-f8eef542f0f5>:46] Epoch 9, train: 0s
I0912 08:36:45.614427  8168 <ipython-input-29-f8eef542f0f5>:48] Total loss: 18.5721, max_margin_loss: 18.5721, ortho_reg: 0.0000
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 43.31it/s]


Aspect 0:
['אלוביץ : 0.9786866', 'הפרה : 0.9785516', 'עבירה : 0.97819847', 'דין : 0.97805583', "אלוביץ' : 0.9779986", 'משפטי : 0.9779612', 'שאול : 0.9779428', 'בזק : 0.97786874', 'מקורב : 0.9775635', 'אביחי : 0.9775218', 'ראש : 0.9775014', 'שוחד : 0.9775002', 'אתר : 0.9774548', 'תיק : 0.97742105', 'יועץ : 0.9773958', 'בעל : 0.9773333', 'יחסים : 0.9773217', 'הימין : 0.97727305', 'עניין : 0.9772569', 'שיחה : 0.9771797']


I0912 08:36:45.925445  8168 <ipython-input-29-f8eef542f0f5>:46] Epoch 10, train: 0s
I0912 08:36:45.926428  8168 <ipython-input-29-f8eef542f0f5>:48] Total loss: 18.4748, max_margin_loss: 18.4748, ortho_reg: -0.0000
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 42.97it/s]


Aspect 0:
['אלוביץ : 0.97757894', 'הפרה : 0.97753257', 'עבירה : 0.9770818', 'דין : 0.9769836', "אלוביץ' : 0.9769039", 'שאול : 0.97687566', 'משפטי : 0.97687197', 'בזק : 0.97669375', 'יחסים : 0.9764892', 'אביחי : 0.9764622', 'מקורב : 0.97641695', 'שוחד : 0.9763664', 'ראש : 0.97635144', 'אתר : 0.97633433', 'תיק : 0.9763096', 'בעל : 0.97623867', 'יועץ : 0.97623354', 'הימין : 0.97622806', 'עניין : 0.9761323', 'חוקר : 0.9761218']


I0912 08:36:46.234426  8168 <ipython-input-29-f8eef542f0f5>:46] Epoch 11, train: 0s
I0912 08:36:46.235427  8168 <ipython-input-29-f8eef542f0f5>:48] Total loss: 18.3906, max_margin_loss: 18.3906, ortho_reg: 0.0000
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 43.31it/s]
I0912 08:36:46.496438  8168 <ipython-input-29-f8eef542f0f5>:46] Epoch 12, train: 0s
I0912 08:36:46.497431  8168 <ipython-input-29-f8eef542f0f5>:48] Total loss: 18.5045, max_margin_loss: 18.5045, ortho_reg: -0.0000
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 49.11it/s]
I0912 08:36:46.731434  8168 <ipython-input-29-f8eef542f0f5>:46] Epoch 13, train: 0s
I0912 08:36:46.732434  8168 <ipython-input-29-f8eef542f0f5>:48] Total loss: 18.4573, max_margin_loss: 18.4573, ortho_reg: -0.0000
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 40.00it/s]

In [36]:
list_aspects=''
for v in desc_list:
    print(v.split(':')[0])
    list_aspects+=v.split(':')[0]+','

אלוביץ 
הפרה 
עבירה 
דין 
אלוביץ' 
שאול 
משפטי 
בזק 
יחסים 
אביחי 
מקורב 
שוחד 
ראש 
אתר 
תיק 
בעל 
יועץ 
הימין 
עניין 
חוקר 


In [37]:
list_aspects

"אלוביץ ,הפרה ,עבירה ,דין ,אלוביץ' ,שאול ,משפטי ,בזק ,יחסים ,אביחי ,מקורב ,שוחד ,ראש ,אתר ,תיק ,בעל ,יועץ ,הימין ,עניין ,חוקר ,"